# This nootbook Extracts Embeddings for tweets using Arabert, then Classifies the embeddings using SVM Classifiers

In [1]:
!pip install bert-for-tf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 677.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30510 sha256=2d592da02ae9ebab10702b2d0973ae09550ac543201e38ca9816928b4d7cfe44
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19455 sha256=9284f71a00162dad6f1c99b49f870f7cf243ad638f24f93351786307a7a17fe6
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7891 sha256=7f4b76e3566e8020399d308d82ad55467287a941c6f0cfa2455e4888e21012ea
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e9087

In [2]:
!pip install transformers
!pip install arabert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=b9963fb9a199dd4d318cfdcb1b5f438a972a040334d0b873265ff21ce9d55c8f
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji


In [3]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [44]:
train = pd.read_csv("train.csv")
test= pd.read_csv("test.csv")

# Extract text and labels
train_texts = train["comment"]
train_labels = train["label"]

test_texts = test["comment"]
test_labels = test["label"]


In [3]:
# Load AraBERT
model_name = "aubmindlab/bert-base-arabertv02-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [4]:
def get_embeddings(samples):
    embeddings = []

    # Move the model to GPU for speeeeeed 
    model.to('cuda')

    for sample in samples:
        # Tokenize the text and move the tensors to GPU
        tokenized_text = tokenizer(sample, padding=True, truncation=True, return_tensors="pt").to('cuda')

        with torch.no_grad():
            outputs = model(**tokenized_text)
            sample_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Move to CPU tensor before converting to numpy bcz the Incompatibility of NumPy with GPU Tensors

        embeddings.append(sample_embedding)

    embeddings = np.vstack(embeddings)  # Create a 2D array
    return embeddings


In [46]:
train_texts= train_texts.to_list()
test_texts= test_texts.to_list()

In [ ]:
# Obtain embeddings for all samples
X_train = get_embeddings(train_texts)
X_test = get_embeddings(test_texts)
y_train = train_labels
y_test = test_labels

In [ ]:
# Train SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

# Predict using the trained classifier
y_pred = svm_classifier.predict(X_test)

In [2]:
print(classification_report(y_test, y_pred))